In [25]:
from GaussianGenerator import GaussianGenerator
import numpy as np
import threading
import communicator
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import socket
import time
from IPython.display import clear_output, display
import os

In [2]:
def GaussianGenerator (n1, n2):
	mean1 = [3, 6]
	cov1 = [[1,0], [0,1]]
	x1 = np.random.multivariate_normal(mean1,cov1, n1)
	y1 = np.ones(n1)
	
	mean2 = [-1,0]
	cov2 = [[2,0],[0,2]]
	x2 = np.random.multivariate_normal(mean2,cov2, n2)
	y2 = -np.ones(n2)
	
	#plt.plot(x1[:,0], x1[:,1], 'x')
	#plt.plot(x2[:,0], x2[:,1], '*')
	#plt.show()
	#plt.savefig("gaussian.png")
	

	X = np.concatenate((x1,x2), axis = 0)
	Y = np.concatenate((y1,y2), axis = 0)
	return X,Y

In [3]:
DEBUG = True
BACKLOG = 5
HEADERSIZE = 10


class Server:

	def __init__(self, port):
		"""
		Server object that can accept request at port=port
		Create a SOCK_STREAM socket and bind at localhost at port
		"""
		self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
		self.s.bind((socket.gethostname(), port))
		self.s.listen(BACKLOG)  # backlog = 5 by default
		self.clientsock = None
		self.clientaddress = None

	def recv(self):
		"""
		Blocking recv. Block to accept and receive.
		:return: msg to be pickled
		"""
		self.clientsock, self.clientaddress = self.s.accept()

		#if DEBUG:
			#print(f"Received message from {self.clientaddress}")

		full_msg = b''
		new_msg = True
		msglen = 0
		while True:  # receive full length
			msg = self.clientsock.recv(16)
			if new_msg:
				#print("new msg len:", msg[:HEADERSIZE])
				msglen += int(msg[:HEADERSIZE])
				new_msg = False
			full_msg += msg
			if len(full_msg) - HEADERSIZE == msglen:
				#print("full msg recvd")
				return pickle.loads(full_msg[HEADERSIZE:])

	def reply(self, msg, port):
		"""
		Reply to msg that we just received from
		:return: num_byte_sent
		"""
		
		loc = (self.clientaddress[0],port)
		print(loc)
		return self.send(loc, msg)

	def get_socket(self):
		return self.s

	def send(self, location: (str, str), msgtosend):
		"""
		Send msg to location
		:param location: specify (address, port) to be sent to
		:param msgtosend: the msg will be pickled
		:return:
		"""
		s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
		s.connect(location)
		msg = pickle.dumps(msgtosend)
		msg = bytes(f"{len(msg):<{HEADERSIZE}}", 'utf-8') + msg
		return s.send(msg)


In [4]:
PORT = 1232
PORT2 = 1233
HOSTADDR = "container1"
serv = communicator.Server(PORT)
loc_model = (HOSTADDR, PORT)
loc_server = (HOSTADDR, PORT2)

In [ ]:

#get dataset
X, Y = GaussianGenerator(100,100)
x_temp = np.ones((200,1))
X = np.concatenate((x_temp,X),axis = 1)

while True:
    enter = input('Hit ENTER to send update:')
    while enter != '':
        enter = input('Hit ENTER to send update:')

    #send request to server for model w
    msg = "Req"
    serv.send(loc_model, msg)

    #wait for model 
    w = serv.recv()
    print(f"Received weights {str(w)}")
    #w = [1,1,1]


    #get sample
    r = np.random.choice(200, 10)
    x = X[r]
    y = Y[r]

    #compute gradient
    g = np.array([0,0,0])
    for i in range(10):
        if ((np.dot(w,x[i])>=0 and y[i]==-1) or (np.dot(w,x[i])<0 and y[i]==1)):
            g = g -np.multiply(y[i],x[i])

    g = g/10
    

    #Byzantine behavior
    mal = input("Is the update malicious? [y/n]")
    if mal== 'y':
        #g = -np.multiply(10,g)
        g = 100*np.random.randn(3)+3
        print(f"Malicious grad is {str(g)}")
    else:
        print(f"Grad computed is {str(g)}")

    #send g to server
    serv.send(loc_server, g)
    time.sleep(3)

Hit ENTER to send update:
Received weights [ 0.06299457  0.01523867 -0.0447077 ]
Is the update malicious? [y/n]n
Grad computed is [-0.2        -1.69522586 -3.08464349]
Hit ENTER to send update:
Received weights [0.07879386 0.14915553 0.19896825]
Is the update malicious? [y/n]n
Grad computed is [ 0.1        -0.14720276  0.16555933]
Hit ENTER to send update:
Received weights [0.07879386 0.14915553 0.19896825]
Is the update malicious? [y/n]n
Grad computed is [ 0.3        -0.20796218  0.39203761]
Hit ENTER to send update:
Received weights [0.07879386 0.14915553 0.19896825]
Is the update malicious? [y/n]n
Grad computed is [ 0.1        -0.14854758  0.15295449]
Hit ENTER to send update:
Received weights [0.06555542 0.16882092 0.17871945]
Is the update malicious? [y/n]n
Grad computed is [ 0.2         0.19529986 -0.00908944]
Hit ENTER to send update:
Received weights [0.06555542 0.16882092 0.17871945]
Is the update malicious? [y/n]n
Grad computed is [ 0.3        -0.18864833  0.46085374]
Hit ENT